In [42]:
import json
import pennylane as qml
import pennylane.numpy as np

# Write any helper functions you need
def Ua(a):
    off_diag = np.sqrt(1 - (a ** 2))
    return np.array([[a, off_diag], [off_diag, -a]])

def S(phi):
    return np.array([[np.exp(1j*phi), 0], [0, np.exp(-1j*phi)]])

def poly(angles, a):
    product = S(angles[0])
    for phi in angles[1:]:
        product = np.dot(product, np.dot(Ua(a), S(phi)))
    return product[0, 0]


def coefficients(angles):
    
    """ This function returns the coefficients associated with the polynomial generated by 
    the QSP routine as a function of the phase angles.
    
    Args:
        - angles (np.array(float)): Array of real numbers containing the four phase angles,
        in reverse order of application.
    
    Returns:       
        - (np.array(complex)): A numpy array containing the coefficients of the polynomial
        generated by QSP, where the first element is the coefficient of the cubic term and
        the second element is for the linear term.
    """
    # Put your code here
    x = poly(angles, 1)
    y = poly(angles, 0.5)
    beta = ((8*y) - x) / 3
    alpha = x - beta
    return np.array([alpha, beta])


In [43]:
import matplotlib.pyplot as plt

In [44]:
coefficients([0.5,0.8,1.0,1.0])

tensor([-0.17079762-2.4084889j, -0.81668215+2.2507432j], requires_grad=True)

In [28]:
np.dot(np.array([[2, 0], [0, 2]]), np.array([[1, 2], [3, 4]]))

tensor([[2, 4],
        [6, 8]], requires_grad=True)

In [45]:
# These functions are responsible for testing the solution.


def run(test_case_input: str) -> str:
    ins = np.array(json.loads(test_case_input))
    coeffs = coefficients(ins)
    outs = [[np.real(elem).numpy(), np.imag(elem).numpy()] for elem in coeffs]
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    print(solution_output, expected_output)

    assert np.allclose(solution_output, expected_output, rtol = 1e-4)


# These are the public test cases
test_cases = [
    ('[0.5,0.8,1.0,1.0]', '[[-0.1707976, -2.4084889], [-0.8166822, 2.2507432]]'),
    ('[-0.20409113, -0.91173829, 0.91173829, 0.20409113]', '[[2.5,0],[-1.5,0]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.5,0.8,1.0,1.0]'...
[[-0.17079761511232194, -2.4084888966742493], [-0.816682154796543, 2.250743202531001]] [[-0.1707976, -2.4084889], [-0.8166822, 2.2507432]]
Correct!
Running test case 1 with input '[-0.20409113, -0.91173829, 0.91173829, 0.20409113]'...
[[2.4999999962490627, -3.700743415417189e-17], [-1.4999999962490629, -4.625929269271485e-17]] [[2.5, 0], [-1.5, 0]]
Correct!
